# Código TP3: Clustering

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
from sklearn.datasets import load_iris

# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html
from sklearn.preprocessing import scale

# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
from sklearn.cluster import KMeans

# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
from sklearn.decomposition import PCA

## Carga de Datasets "Crabs"

In [2]:
crabs = pd.read_csv("Crabs.csv", sep=',')
crabs = crabs.drop(columns=["rownames", "index"])
print(crabs)

crabs_numpy = crabs.to_numpy()
print(crabs_numpy)

    sp sex    FL    RW    CL    CW    BD
0    B   M   8.1   6.7  16.1  19.0   7.0
1    B   M   8.8   7.7  18.1  20.8   7.4
2    B   M   9.2   7.8  19.0  22.4   7.7
3    B   M   9.6   7.9  20.1  23.1   8.2
4    B   M   9.8   8.0  20.3  23.0   8.2
..  ..  ..   ...   ...   ...   ...   ...
195  O   F  21.4  18.0  41.2  46.2  18.7
196  O   F  21.7  17.1  41.7  47.2  19.6
197  O   F  21.9  17.2  42.6  47.4  19.5
198  O   F  22.5  17.2  43.0  48.7  19.8
199  O   F  23.1  20.2  46.2  52.5  21.1

[200 rows x 7 columns]
[['B' 'M' 8.1 ... 16.1 19.0 7.0]
 ['B' 'M' 8.8 ... 18.1 20.8 7.4]
 ['B' 'M' 9.2 ... 19.0 22.4 7.7]
 ...
 ['O' 'F' 21.9 ... 42.6 47.4 19.5]
 ['O' 'F' 22.5 ... 43.0 48.7 19.8]
 ['O' 'F' 23.1 ... 46.2 52.5 21.1]]


## Carga de Datasets "Lampone"

In [ ]:
lampone = pd.read_csv("Lampone.csv", sep=',')
print(lampone)

lampone_numpy = lampone.to_numpy()
print(lampone_numpy)

    anno     m33   m34   m35   m36  m40   m41   m42    m43   m44  ...  m183  \
0   2006   32.24  0.27  0.06  0.01  0.0  0.30  0.02   2.05  0.11  ...  0.00   
1   2006   33.41  0.34  0.05  0.01  0.0  0.12  0.02  10.42  0.25  ...  0.00   
2   2006   31.93  0.32  0.05  0.01  0.0  0.09  0.02   9.32  0.24  ...  0.00   
3   2006   43.62  0.45  0.08  0.02  0.0  0.11  0.02  12.67  0.31  ...  0.00   
4   2006   47.08  0.47  0.09  0.02  0.0  0.20  0.03   3.19  0.12  ...  0.00   
5   2006   21.38  0.20  0.03  0.00  0.0  0.06  0.01  12.01  0.27  ...  0.00   
6   2006   37.20  0.38  0.06  0.01  0.0  0.07  0.02   3.45  0.14  ...  0.00   
7   2006   57.00  0.51  0.08  0.00  0.0  0.08  0.02   2.63  0.09  ...  0.00   
8   2006   13.31  0.16  0.02  0.00  0.0  0.14  0.02  11.92  0.26  ...  0.00   
9   2006  122.75  1.05  0.17  0.01  0.0  0.11  0.02   8.67  0.24  ...  0.00   
10  2006   37.70  0.37  0.05  0.00  0.0  0.09  0.02  19.00  0.39  ...  0.00   
11  2006   39.47  0.38  0.06  0.01  0.0  0.15  0.02 

## Métodos de Comparación de Resultados

In [3]:
# Comentario: para comparar dos soluciones de clustering o una de ellas
#  contra las clases originales se suele usar una tabla, como por ejemplo:

def comparar_tabla(clusters1, clusters2):

    # Suponemos a los clusters numerados desde 0
    nclusters1 = np.max(clusters1) + 1
    nclusters2 = np.max(clusters2) + 1

    conf_matrix = np.zeros((nclusters1, nclusters2), dtype=int)

    for i in range(len(clusters1)):
        conf_matrix[clusters1[i], clusters2[i]] += 1

    print(conf_matrix)
    return conf_matrix


# pero se puede optimizar el match entre los dos clusterings, para hacer
# mejor la tabla, usando:

def comparar_tabla_optim(clusters1, clusters2):

    # Suponemos a los clusters numerados desde 0
    nclusters1 = np.max(clusters1) + 1
    nclusters2 = np.max(clusters2) + 1

    conf_matrix = np.zeros((nclusters1, nclusters2), dtype=int)

    for i in range(len(clusters1)):
        conf_matrix[clusters1[i], clusters2[i]] += 1

    # minimum weight matching, optimize the matching between a cluster
    # in clusters1 with one or more clusters in clusters2
    _, col_ind = linear_sum_assignment(-conf_matrix)
    optimized_conf_matrix = conf_matrix[:, col_ind]

    print(optimized_conf_matrix)
    return optimized_conf_matrix



## Score de Estabilidad de Soluciones de Clustering

In [4]:
# Código de ejemplo de como calcular el score de estabilidad de dos soluciones
# de clustering usando Iris

data = load_iris()
X = data.data
n = X.shape[0]

# Fijar el número de clusters
k = 3

# Crear dos muestras aleatorias del 90% de los datos
ind1 = np.random.choice(n, int(0.9 * n), replace=False)
ind2 = np.random.choice(n, int(0.9 * n), replace=False)

# Aplicar K-Means a ambas muestras
cc1 = KMeans(n_clusters=k, n_init=10, random_state=0).fit(X[ind1]).labels_
cc2 = KMeans(n_clusters=k, n_init=10, random_state=0).fit(X[ind2]).labels_

# Reinsertar los clusters en un vector de longitud n, con 0 en los puntos
# no seleccionados
v1 = np.zeros(n, dtype=int)
v2 = np.zeros(n, dtype=int)

# Se suman 5 a las etiquetas para permitir el truco de la raíz cuadrada
v1[ind1] = cc1 + 5
v2[ind2] = cc2 + 5

# Crear la matriz de similitud: 1 si están en el mismo cluster,
# -1 si están en distinto, 0 si alguno falta
def compute_similarity_matrix(v):
    a = np.sqrt(np.outer(v, v))
    m = np.divide(a, -a, out=np.zeros_like(a), where=a != 0) + 2 * (a == np.round(a))
    return m

m1 = compute_similarity_matrix(v1)
m2 = compute_similarity_matrix(v2)

# Calcular el score de estabilidad
validos = np.sum((v1 * v2) > 0)
if validos > 1:
    score = np.sum((m1 * m2)[np.triu_indices(n, k=1)] > 0) / (validos * (validos - 1) / 2)
else:
    score = 0  # Evita división por cero

print(score)

1.2202479338842975


# Ejercicio 1

In [ ]:
X = crabs.drop(columns=["sp", "sex"])
y = crabs[["sp", "sex"]]
print(X.shape)
print(y.shape)

(200, 5)
(200, 2)
